# Conv-adapter (residual) experiments with different Factors

In [1]:
import sync_wandb_run
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import utils as pu
from plot import create_plot_with_values
from plot_pretty import create_ultimate_plotly_plot, generate_scatter_plot, generate_scatter_plot_accuracy, generate_scatter_plot_gridscore
import plotly_utils

In [2]:
entity = "raphaelmaser"  # Your W&B username or team
project = "Improved-Interpretability-and-Concepts"  # Your W&B project name
df = sync_wandb_run.load_wandb_runs()
df.count()  # Display the number of runs loaded
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

val_accuracy = "Validation Accuracy"
grid_score = "PG Score"
frozen_layers = "config_define_frozen_layers"
trainable_params = "Trainable Parameters (M)"
figsize=(10,4)


In [3]:
def rename_model_names(df):
    """Rename model names in the DataFrame for better readability."""
    df = df.replace({
        "Model": {
            "B=1_pretrained_CE_bcosified": "B=2 (Bcosified Model)",
            "B=1_pretrained_CE-parallel-adapters-B=2_B=2": "B=2 Model with B=2 Adapter",
            "B=1_pretrained_CE-parallel-residual-adapters-B=2_B=2": "B=2 Model with B=2 Adapter",
            "B=1_pretrained_CE_conv-wise-adapter_B=2": "B=2 Adapter",
        }
    })
    return df

In [4]:
conv_adapter = df[df['group'] == "adapter-factor-conv-wise"]
conv_adapter = conv_adapter[conv_adapter["config_conv_wise_adapter_mode"] == "in_channel"]
bcosified_model = df[df['id'] == "29kcufiq"]
conv_adapter = pd.concat([conv_adapter, bcosified_model], ignore_index=True)  # Add the Bcosified model to the DataFrame
# conv_wo_batchnorm = conv_wo_batchnorm[conv_wo_batchnorm["id"].isin(["sbpfkwct", "dv8y7o8k", "yheexwdv", "zf9oo2l6", "xka2n8rp"])]  # Filter for specific runs
# conv_wo_batchnorm = rename(conv_wo_batchnorm)  # Rename model names for better readability
conv_adapter.reset_index(drop=True, inplace=True)  # Reset index after filtering
print(f"Number of runs in layerwise_freezing group: {len(conv_adapter)}")

#Renaming
conv_adapter = pu.rename(conv_adapter).sort_values(by=val_accuracy, ascending=False)
conv_adapter = rename_model_names(conv_adapter)  # Rename model names for better readability
# conv_adapter = pu.add_trainable_layers_column(conv_adapter, frozen_layers_col=frozen_layers, new_col_name=trainable_layers)
conv_adapter = pu.rename_adapter_factor_experiments(conv_adapter)  # Rename columns for adapter factor experiments

# Some stats
for name in conv_adapter["Model"].unique(): print(name)  # Print each model name
display(conv_adapter)


Number of runs in layerwise_freezing group: 7
B=2 (Bcosified Model)
B=2 Adapter


id                  Model     state        entity  \
6  29kcufiq  B=2 (Bcosified Model)  finished  raphaelmaser   
5  rxkz02pb            B=2 Adapter  finished  raphaelmaser   
0  xwerrpal            B=2 Adapter  finished  raphaelmaser   
3  a9fogf2f            B=2 Adapter  finished  raphaelmaser   
4  8o5ogpzm            B=2 Adapter   crashed  raphaelmaser   
2  iyflqbic            B=2 Adapter  finished  raphaelmaser   
1  fmbc50w0            B=2 Adapter  finished  raphaelmaser   

                                  project            created_at  updated_at  \
6  Improved-Interpretability-and-Concepts  2025-06-30T09:25:37Z         NaN   
5  Improved-Interpretability-and-Concepts  2025-07-23T17:02:51Z         NaN   
0  Improved-Interpretability-and-Concepts  2025-07-23T17:27:42Z         NaN   
3  Improved-Interpretability-and-Concepts  2025-07-23T17:45:25Z         NaN   
4  Improved-Interpretability-and-Concepts  2025-07-03T09:37:56Z         NaN   
2  Improved-Interpretability-and-Concepts  2025-07-03T09:38:22Z         NaN   
1  Improved-Interpretability-and-Concepts  2025-07-03T09:38:52Z         NaN   

  tags notes                                                url  \
6  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
5  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
0  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
3  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
4  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
2  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
1  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   

                      group  job_type  sweep                   synced_at  \
6       adapter-experiments       NaN    NaN  2025-07-24T10:44:04.193189   
5  adapter-factor-conv-wise       NaN    NaN  2025-07-24T10:49:21.339285   
0  adapter-factor-conv-wise       NaN    NaN  2025-07-24T10:49:22.223335   
3  adapter-factor-conv-wise       NaN    NaN  2025-07-24T10:49:22.997549   
4  adapter-factor-conv-wise       NaN    NaN  2025-07-24T10:44:38.453222   
2  adapter-factor-conv-wise       NaN    NaN  2025-07-24T10:44:39.252469   
1  adapter-factor-conv-wise       NaN    NaN  2025-07-24T10:44:40.335352   

   config_b  config_lr config_loss  config_seed config_tags  \
6       2.0      0.001  UniformBCE         42.0         NaN   
5       2.0      0.001  UniformBCE         42.0         NaN   
0       2.0      0.001  UniformBCE         42.0         NaN   
3       2.0      0.001  UniformBCE         42.0         NaN   
4       2.0      0.001  UniformBCE         42.0         NaN   
2       2.0      0.001  UniformBCE         42.0         NaN   
1       2.0      0.001  UniformBCE         42.0         NaN   

               config_group             config_model  config_epochs  \
6       adapter-experiments  BcosResNet50_bottleneck             30   
5  adapter-factor-conv-wise  BcosResNet50_bottleneck             30   
0  adapter-factor-conv-wise  BcosResNet50_bottleneck             30   
3  adapter-factor-conv-wise  BcosResNet50_bottleneck             30   
4  adapter-factor-conv-wise  BcosResNet50_bottleneck             30   
2  adapter-factor-conv-wise  BcosResNet50_bottleneck             30   
1  adapter-factor-conv-wise  BcosResNet50_bottleneck             30   

  config_adapter  config_compile config_dataset  config_vit_dim  \
6            NaN            True   TinyImagenet             NaN   
5            NaN            True   TinyImagenet          1536.0   
0            NaN            True   TinyImagenet          1536.0   
3            NaN            True   TinyImagenet          1536.0   
4            NaN            True   TinyImagenet           768.0   
2            NaN            True   TinyImagenet           768.0   
1            NaN            True   TinyImagenet           768.0   

  config_use_lora config_act_layer  config_b_adapter  \
6             NaN             relu               2.0   
5         

In [5]:
# # Move legend outside the plot
# plt.tight_layout()
# plt.show()

# plt.figure(figsize=figsize)
# ax = sns.scatterplot(
#     data=conv_adapter,
#     y=val_accuracy,
#     x=trainable_params,
#     hue="Model",  # Color points by name
#     # s=100  # Make points larger
# )
# pu.add_val_to_plot(ax)
# plt.ylim(0.6, 0.9)
# # Move legend outside the plot
# plt.tight_layout()
# plt.show()

generate_scatter_plot_accuracy(
    data=conv_adapter,
    x=trainable_params,
    y=val_accuracy,
    color="Model",
    scale_factor=2,
    title="Conv-Adapter (Convolution)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

In [6]:
generate_scatter_plot_gridscore(
    data=conv_adapter,
    x=trainable_params,
    y=grid_score,
    color="Model",
    scale_factor=2,
    title="Conv-Adapter (Convolution)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)